<a href="https://colab.research.google.com/github/tonyjan02/idealnba/blob/main/cis1920final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
# importing necessary libraries
import json
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from google.colab import drive
from sklearn.model_selection import train_test_split

Data Pre-processing/Cleaning

Creating ML Model

Tuning Model

Presenting Findings